#  高级模型训练

在迄今为止的教程中，我们遵循了一个简单的训练模型的程序：加载一个数据集，创建一个模型，调用 `fit()`，评估模型，最后用于自己调用。对于一个例子来说，这很好，但在真正的机器学习项目中，这个过程通常更复杂。在本教程中，我们将看到一个更真实的训练模型的工作流程。

## Colab

本教程和目录中的其余部分都是在 Google colab 中完成。如果您想在 colab 中打开此笔记本，您可以点击以下链接。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BioGavin/deepchem/blob/master/examples/tutorials-zh/Advanced_Model_Training.ipynb)

## 设置

要在 Colab 内运行 DeepChem，你需要运行以下安装命令。当然，如果你愿意，你可以在本地运行这个教程。在这种情况下，不要运行这些单元，因为它们将再次在你的本地机器中下载和安装 DeepChem。


In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

## 超参数优化

让我们从加载 HIV 数据集开始。它根据是否抑制艾滋病毒的复制对4万多个分子进行分类。

In [1]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', split='scaffold')
train_dataset, valid_dataset, test_dataset = datasets

现在让我们在上面训练一个模型。我们将使用 `MultitaskClassifier`，它只是一个密集层的堆栈。但这仍然有很多超参数的选择，应该设置多少个层？每个层应该有多宽？我们应该使用什么丢弃率？什么样的学习率？

这些都被称为超参数。选择它们的标准方法是尝试很多值，在训练集上训练每个模型，并在验证集上评估。这能让我们看到哪些参数效果最好。

你可以手动来做，但通常让计算机自动为你做这件事更容易。DeepChem 提供了一个超参数优化算法的选择，在 `dc.hyper` 包中可以找到这些算法。对于这个例子，我们将使用`GridHyperparamOpt`，这是最基本的方法。我们只是给它一个每个超参数的选项列表，它详尽地尝试它们的所有组合。

选项列表是由我们提供的 `dict` 定义的。对于模型的每个参数，我们提供了一个可以尝试的值的列表。在这个例子中，我们考虑了三组可能的隐藏层：一个宽度为 500 的单层，一个宽度为 1000 的单层，或者两个宽度为 1000 的层。我们还考虑了两种丢弃率（20%和50%）和两种学习率（0.001和0.0001）。

In [2]:
params_dict = {
    'n_tasks': [len(tasks)],
    'n_features': [1024],
    'layer_sizes': [[500], [1000], [1000, 1000]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}
optimizer = dc.hyper.GridHyperparamOpt(dc.models.MultitaskClassifier)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train_dataset, valid_dataset, metric, transformers)

`hyperparam_search()` 返回三个参数：它找到的最佳模型、该模型的超参数、以及每个模型的验证分数的完整列表。让我们看一下最后一个参数。

In [3]:
all_results

{'_dropouts_0.200000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.759624393738977,
 '_dropouts_0.200000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7680791323731138,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7623870149911817,
 '_dropouts_0.500000_layer_sizes[500]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7552282358416618,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7689915858318636,
 '_dropouts_0.200000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7619292572996277,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.7641491524593376,
 '_dropouts_0.500000_layer_sizes[1000]_learning_rate_0.000100_n_features_1024_n_tasks_1': 0.7609877155594749,
 '_dropouts_0.200000_layer_sizes[1000, 1000]_learning_rate_0.001000_n_features_1024_n_tasks_1': 0.770716980207721,
 '_dropout

我们可以看到一些一般的模式。使用两层较大的学习率，效果并不理想，似乎更深的模型需要一个更小的学习率。我们还看到，20%的丢弃率通常比50%好用。一旦我们根据这些观察结果缩小了模型的超参范围，所有的验证分数都会非常接近，可能接近到剩余的变化主要是噪音。我们使用剩下的超参数集似乎没有什么区别，所以让我们任意选择一个宽度为 1000 的单层，学习率为 0.0001。

## 提前停止 (Early Stopping)

还有一个重要的超参数我们还没有考虑：我们训练模型的时间有多长。 `GridHyperparamOpt` 对每个模型进行固定的、相当少的 epochs 的训练。这不一定是最好的数字。

你可能期望你训练的时间越长，你的模型就越好，但这通常不会如此。如果你训练的时间太长，模型通常会开始对训练集的不相关细节进行过度拟合。你可以知道什么时候会发生这种情况，当验证集的分数不再增加，甚至可能减少，而训练集的分数继续提高的时候。

幸运的是，我们不需要为不同的步骤数训练很多不同的模型来确定最佳的数字。我们只需训练一次，监测验证分数，并保持任何一个参数都能使其最大化。这被称为 "early stopping"。 DeepChem 的 `ValidationCallback` 类可以自动为我们做这个。在下面的例子中，我们让它每 1000 个训练步骤计算验证集的 ROC AUC。如果你添加 `save_dir` 参数，它也会将最佳模型参数的副本保存到磁盘。

In [4]:
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=0.0001)
callback = dc.models.ValidationCallback(valid_dataset, 1000, metric)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.759757
Step 2000 validation: roc_auc_score=0.770685
Step 3000 validation: roc_auc_score=0.771588
Step 4000 validation: roc_auc_score=0.777862
Step 5000 validation: roc_auc_score=0.773894
Step 6000 validation: roc_auc_score=0.763762
Step 7000 validation: roc_auc_score=0.766361
Step 8000 validation: roc_auc_score=0.767026
Step 9000 validation: roc_auc_score=0.761239
Step 10000 validation: roc_auc_score=0.761279
Step 11000 validation: roc_auc_score=0.765363
Step 12000 validation: roc_auc_score=0.769481
Step 13000 validation: roc_auc_score=0.768523
Step 14000 validation: roc_auc_score=0.761306
Step 15000 validation: roc_auc_score=0.77397
Step 16000 validation: roc_auc_score=0.764848


0.8040038299560547

## 学习率策略

在上面的例子中，我们在整个训练过程中使用一个固定的学习率。在某些情况下，在训练期间改变学习率效果更好。为了在 DeepChem 中做到这一点，我们只需指定一个 `LearningRateSchedule` 对象，而不是一个 `learning_rate` 参数的数字。在下面的例子中，我们使用一个按指数下降的学习率，它从 0.0002 开始，然后在每 1000 步后被乘以 0.9。

In [5]:
learning_rate = dc.models.optimizers.ExponentialDecay(0.0002, 0.9, 1000)
model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                                      n_features=1024,
                                      layer_sizes=[1000],
                                      dropouts=0.2,
                                      learning_rate=learning_rate)
model.fit(train_dataset, nb_epoch=50, callbacks=callback)

Step 1000 validation: roc_auc_score=0.736547
Step 2000 validation: roc_auc_score=0.758979
Step 3000 validation: roc_auc_score=0.768361
Step 4000 validation: roc_auc_score=0.764898
Step 5000 validation: roc_auc_score=0.775253
Step 6000 validation: roc_auc_score=0.779898
Step 7000 validation: roc_auc_score=0.76991
Step 8000 validation: roc_auc_score=0.771515
Step 9000 validation: roc_auc_score=0.773796
Step 10000 validation: roc_auc_score=0.776977
Step 11000 validation: roc_auc_score=0.778866
Step 12000 validation: roc_auc_score=0.777066
Step 13000 validation: roc_auc_score=0.77616
Step 14000 validation: roc_auc_score=0.775646
Step 15000 validation: roc_auc_score=0.772785
Step 16000 validation: roc_auc_score=0.769975


0.22854619979858398

# 恭喜！是时候加入社区了！

恭喜您完成本教程笔记本！如果您喜欢本教程并希望继续使用 DeepChem，我们鼓励您完成本系列的其余教程。您还可以通过以下方式帮助 DeepChem 社区：

## 在 [GitHub](https://github.com/deepchem/deepchem) 上为 DeepChem 点亮小星星
这有助于大家建立对 DeepChem 项目和我们正在尝试构建的开源药物发现工具的共识。

## 加入 DeepChem Gitter
DeepChem [Gitter](https://gitter.im/deepchem/Lobby) 聚集了许多对生命科学深度学习感兴趣的科学家、开发人员和爱好者，欢迎加入！